![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : "WNTEfrjfUygDMK5_8eW-TMGYGUgNgxw2aor4bWsb0Wit"
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [2]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [3]:
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 600,
    "stop_sequences": ["<end_json>"],
    "temperature": 0.6,
    "top_k": 50,
    "top_p": 1,
    "repetition_penalty": 1.09
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [4]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [5]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


In [6]:
noun_type =  """ضمير مفرد""" 
questions_count =  """5 أسئلة""" 

In [7]:
prompt_input = f"""لنلعب لعبة باللغة العربية وهي تأليف جملة اسمية بسيطة. المطلوب إكمال جملة تبدأ بأحد أشكال المبتدأ.

أشكال المبتدأ في الجملة الاسمية:
أمثلة عن أشكال المبتدأ
- الاسم الصريح المعرف بـأل التعريف،حالة المفرد،مثال:
1- الصبر مفتاح الفرج.
2- الوطن أعز ما نملك.
3- الحقيقة واضحة.

- الاسم الصريح المعرف بـأل التعريف،حالة المثنى،مثال:
1- الطالبتان متفوقتان.
2- المهندسان مسافران لحضور المؤتمر
3- العالمان مبتكران

- الاسم الصريح المعرف بـأل التعريف،حالة جمع المؤنث السالم،مثال:
- السيارات سريعة.
- الفنانات موهوبات

- الاسم الصريح المعرف بـأل التعريف،حالة جمع المذكر السالم،مثال:
1- اللاعبون محترفون.
2- المسافرون متعبون.
3- المؤمنون متوحدون فيما بينهم.
4- الفلاحون بارعون في الزراعة.

- الاسم الصريح المعرف بـأل التعريف،حالة جمع التكسير،مثال:
1- المدن صاخبة.
2- الوجوه مبتسمة.
3- الثمار لذيذة.
4- الجبال شاهقة.

-الاسم الصريح كاسم علم:
1- زيد ذكي.
2- فاطمة جميلة.
3- عمر كريم.
4- ليلى شاعرة.
5- خالد رياضي.
6- هند طبيبة.

- حالة الضمير المفرد:
1- أنت مهندس ماهر.
2- هو ابن بار.
3- هي ابنة مطيعة.
4- أنا قارئ نهم.

- حالة الضمير المثنى:
1- أنتما صديقان حميمان.
2- هما أخوان متعاونان.
3- أنتما مهندستان مبتكرتان.
4- هما طالبتان مجدتان.

- الضمير الجمع:
1- نحن طلاب مجتهدون.
2- أنتم مهندسون أذكياء.
3- هم أطباء متخصصون.
4- هن بنات بارات.
5- نحن قراء متشوقون.
6- أنتن فتيات حسناوات.

- اسم الإشارة:
1- هذا كتاب جميل.
2- هذه قصة مشوقة.
3- هؤلاء طلاب مجتهدون.
4- أولئك رجال شجعان.
5- تلك سماء صافية.
6- هذا جبل.
7- هذه حديقة.


توصيف اللعبة:
سنساعد المتعلم على فهم جميع أشكال المبتدأ.
ألّف جملة فيها فراغ واحد في البداية. هذا الفراغ هو المبتدأ.
ألّف الجملة بحيث يكون للفراغ أكثر من إجابة صحيحة محتملة.
اختر إجابة صحيحة منطقية ومتعلقة بموضوع الجملة.
لا تكرر أي خرج


تنسيق الخرج:
قدم لي الخرج بتنسيق JSON، حيث يكون لكل سؤال كائن يحتوي على الحقول التالية:
question: وهو الجملة ذات الفراغ.
correct_answer: وهي الكلمة الصحيحة التي تكمل الجملة.
help_questions: وهي قائمة من ثلاثة أسئلة هدفها مساعدة المتعلم على تخمين الإجابة الصحيحة.اجعل واحد من الأسئلة المساعدة سؤال يخبرنا بعدد أحرف الإجابة الصحيحة ثم يصف وظيفة أو شكل أو مواصفات الإجابة الصحيحة إن أمكن ذلك.
grammar: وهو شرح بسيط للقاعدة النحوية.

Input: اكتب جملة فيها فراغ واحد بحيث تحذف المبتدأ ،يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة. اعتمد على القاعدة التالية لتوليد الخرج:
المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف. ولِّد سؤالين.
Output: <start_json>
[
  {{
	"question": "... مغلق.",
	"correct_answer": "الباب",
	"help_questions": ["ما هو الشيء الذي عادة ما يكون له قفل ويمكن فتحه وإغلاقه؟", "تما هو الشيء الذي ندخله منه إلى الغرفة ونخرجه منه؟", "هو جسم خشبي صلب، له مقبض وعادةً ما يكون لونه بني، فما هو"],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف."
  }},

  {{
	"question": "... من أهم مصادر الطاقة المتجددة.",
	"correct_answer": "الطاقة الشمسية",
	"help_questions": [
"ما هو المصدر الذي يستخدم أشعة الشمس لتوليد الكهرباء؟",
"ما هو المصدر الذي يعتبر صديق للبيئة ومستدام؟",
"ما هو المصدر الذي يستخدم الألواح الشمسية لاستغلال الطاقة؟",
],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف."
  }},
]
<end_json>


Input: اكتب جملة فيها فراغ واحد بحيث تحذف المبتدأ ،يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة. اعتمد على القاعدة التالية لتوليد الخرج:
المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل ضمير مفرد.
Output: <start_json>
[
  {{
	"السؤال": "... معلمة متفانية.",
	"correct_answer": "هي",
	"help_questions": ["إذا قلنا أن فاطمة معلمة متفانية، ما هو الضمير الذي يمكن أن يحل محل اسم فاطمة؟", 
"تخيل أنك تتحدث عن صديقتك، كيف تصفها؟ استخدم الضمير المناسب", 
"ما هو الضمير الذي يشير إلى الاسم المفرد المؤنث؟"],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل ضمير."
  }},
]
<end_json>

Input: اكتب جملة فيها فراغ واحد بحيث تحذف المبتدأ ،يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة. اعتمد على القاعدة التالية لتوليد الخرج:
المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم إشارة.
Output: <start_json>
[
  {{
	"question": "... علم بلادي.",
	"correct_answer": "هذا",
	"help_questions": [
ما هي الكلمة التي نستخدمها للإشارة إلى شيء قريب منا ونريده؟",
      "عندما نريد التعبير عن فخرنا بشيء يمثل بلدنا، ما هي الكلمة التي نستخدمها للإشارة إليه؟",
      "كلمة مؤلفة من ثلاثة أحرف، وهي الاسم المذكر من اسم الإشارة (هي)؟",
],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم إشارة."
  }},
]
<end_json>

Input: اكتب جملة فيها فراغ واحد بحيث تحذف المبتدأ ،يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة. اعتمد على القاعدة التالية لتوليد الخرج:
المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف. ولِّد سؤالين.
Output:  <start_json>
[
  {{
	"question": "... مدينة جميلة.",
	"correct_answer": "القاهرة",
	"help_questions": [
"ما هي عاصمة مصر؟",
"أي دولة تعتبر ملتقى الحضارات القديمة والحديثة؟",
"ما هي المدينة المعروفة بتاريخها العريق وأسواقها النابضة بالحياة؟",
],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف."
  }},
  {{
	"question": "... أسرع حيوان.",
	"correct_answer": "الظبي ",
	"help_questions": ["ما هو الحيوان الذي يتميز بسرعة تفوق معظم الحيوانات الأخرى؟",
"اسم حيوان يبدأ بحرف الظاء ويعتبر أسرع الكائنات الحية؟",
"حيوان يتميز بالرشاقة والسرعة العالية، وقد ورد ذكره في القرآن الكريم",
],
	"grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل اسم ظاهر معرف بأل التعريف."
  }},
]
<end_json>

Input: اكتب جملة فيها فراغ واحد بحيث تحذف المبتدأ ،يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة. اعتمد على القاعدة التالية لتوليد الخرج:
المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل {noun_type}. ولِّد {questions_count}.
Output:"""

## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [8]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=False)
print(generated_response)


Submitting generation request...
 <start_json>
[
  {
    "question": "... طالبٌ نشيط.",
    "correct_answer": "هو",
    "help_questions": [
      "إذا كنا نتحدث عن طالب مجتهد ومثابر، ماذا نقول عنه؟",
      "عند الحديث عن شخص يتميز بالنشاط والاجتهاد، ماذا نستخدم للإشارة إليه؟",
      "ما هو الضمير المفرد الذي يعبر عن المفرد المذكر؟",
      "في اللغة العربية، كيف نشير إلى شخص مذكر بواسطة ضميره؟",
      "ما هو الضمير المناسب لاستخدامه مع كلمة طالب لوصف نشاطه واجتهاده؟",
    ],
    "grammar": "المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل ضمير مفرد."
  },
  {
    "question": "... فتاة موهوبة.",
    "correct_answer": "هي",
    "help_questions": [
      "إذا كنا نتحدث عن فتاة تمتلك مهارات ومواهب مميزة، ماذا نقول عنها؟",
      "عند وصف فتاة تتمتع بمواهب وقدرات خاصة، ماذا نستخدم للإشارة إليها؟",
      "ما هو الضمير المفرد المستخدم للتعبير عن الفتاة الأنثى؟",
      "كيف نشير إلى فتاة باستخدام ضميرها في اللغة العربية؟",
      "ما هو الضمير المناسب لاستخدامه مع كلمة فتاة لوصف مواهبها 

In [12]:
import json
json_data = generated_response.replace("<start_json>", "").replace("<end_json>", "").strip()
data = json.loads(json_data)
print("json ", data)

JSONDecodeError: Expecting value: line 11 column 5 (char 429)

In [18]:
num_words = """أكثر من كلمتين"""
question = f"""املأ الفراغ التالي ب{num_words} بالمبتدأ المناسب."""

In [19]:
question

'املأ الفراغ التالي بأكثر من كلمتين بالمبتدأ المناسب.'

In [1]:
def format_questions_count_string(count=1): 
    if count == 1: 
        return "سؤال واحد"
    elif count == 2: 
        return "سؤالين"
    elif count > 2 and count <= 10: 
        return f"""{count} أسئلة"""
    elif count > 10: 
        return f"""{count} سؤال"""

In [3]:
print(format_questions_count_string(12))


12 سؤال
